In [0]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import requests

import os
import psycopg2

is_last = False

In [0]:
def log(is_last, msg):
    now = datetime.now()
    dt_string = now.strftime("%Y-%m-%d %H:%M:%S")
    
    with open("pollution_log.txt", "a") as f:
        if is_last == False:           
            f.write("\n" + msg + " " + "log: " + dt_string)
        else:
            f.write("\n" + msg + " " + "log: " + dt_string + "\n" + "-------------" + "\n")
        

In [0]:
try:
    connection = psycopg2.connect("dbname=pollutionProject user=postgres password=*******")
    
    cursor = connection.cursor()
    
    select_max_date = """select max(local) from pollutiondata;"""
     
    cursor.execute(select_max_date)
    
    max_date = cursor.fetchall()
    max_date_add = max_date[0][0] + timedelta(hours=1)
    date_from = max_date_add.strftime("%Y-%m-%dT%H:%M:%S.000Z")
    
    connection.commit()
    
    msg = "Pollutiondata table successfully opened. Max date selected: " + date_from
    log(is_last, msg)
    
except (Exception, psycopg2.Error) as error:
    
    is_last = True
    msg = "Error while connecting to PostgreSQL: " + error
    log(is_last, msg)
    
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            #print("PostgreSQL connection is closed")
date_from

'2019-07-25T21:00:00.000Z'

In [0]:
#API call

country = 'HU'
page = 1
data_full = []
api = True

msg = "Collecting data from the API."
log(False, msg)


while api == True:
      
    try:
        url = 'https://api.openaq.org/v1/measurements/?country=' + country + '&limit=10000&page=' + str(page) + '&date_from=' + str(date_from)
        r = requests.get(url)

        log(False, url)
        print(url)
        data_json = r.json()

        data = data_json["results"]

        if len(data) == 0:
          break


        data_full = data_full + data

        page += 1

    
    except:
        msg = "Error during the request: " + "status code: " + str(r.status_code)
        log(True, msg)
        #print("page: " + str(page) + " status code: " + str(r.status_code))
        
msg = "Collected: " + str(len(data_full)) + " rows"
log(False, msg)

https://api.openaq.org/v1/measurements/?country=HU&limit=10000&page=1&date_from=2019-07-25T21:00:00.000Z
https://api.openaq.org/v1/measurements/?country=HU&limit=10000&page=2&date_from=2019-07-25T21:00:00.000Z


In [0]:
#Data cleaning

#print("idő, long-lat szétválkasztása")
df = pd.DataFrame(data_full)
df[["latitude", "longitude"]] = df["coordinates"].apply(pd.Series)
df[["utc", "local"]] = df["date"].apply(pd.Series)
new_data = df.drop(["coordinates", "date", "utc", "city", "unit"], axis=1)

msg = "Data cleaning."
log(False, msg)

#pivoting
#print("pivoting")
new_data = new_data.pivot_table(index = ["local", "country", "location", "latitude", "longitude"], columns="parameter")

new_data = pd.DataFrame(new_data.to_records())

new_data.columns = [hdr.replace("('value', ", "").replace(")", "").replace("'", "") \
                     for hdr in new_data.columns]

msg = "Pivoting."
log(False, msg)

#addig upload date 
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")
new_data["upload_date"] = dt_string

#save the new file with the name of the max date

#print("mentés")
max_date = max(new_data.local).split("T", 1)[0]
file_name = "HU_" + str(max_date) + ".csv"

save_path = './pollution_data/'
new_data.to_csv(save_path + file_name, index=False, sep=';')

msg = "Saving the new file as: " + file_name
log(False, msg)

In [0]:
df_values = new_data[['local', 'country', 'location', 'latitude', 'longitude', 'co', 'no2', 'o3', 'pm10', 'pm25', 'so2', 'upload_date']]
df_values_lst = [tuple(x) for x in df_values.values]
df_insert = tuple(df_values_lst)

msg = "Tranforming the data into proper format for inserting."

try:
    connection = psycopg2.connect("dbname=pollutionProject user=postgres password=M4jom1993")
    
    cursor = connection.cursor()
    
    insert = """insert into pollutiondata (Local, Country, Location, Latitude, Longitude, Co, No2, O3, Pm10, Pm25, So2, upload_date)
    values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
    
    cursor.executemany(insert, df_insert)
          
    connection.commit()
    
    msg = "Data inserted successfully into the database!"
    log(False, msg)
    #print("Data inserted successfully! Yeah!")
    
except (Exception, psycopg2.Error) as error :
    #print ("Error while connecting to PostgreSQL", error)
    msg = "Error while connecting to PostgreSQL. Error: " + error
    log(True, msg)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            msg = "Database has been updated. It's time to close!"
            log(True, msg)
            #print("PostgreSQL connection is closed")